In [1]:
#import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm, tqdm_notebook
#from joblib import Parallel, delayed
#from skimage.io import imread
#from skimage.transform import resize
#import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from tqdm import tqdm

# for reading and displaying images

#%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# torchvision for pre-trained models
from torchvision import models
from torchsummary import summary


import shutil, sys   
#from google.colab.patches import cv2_imshow
import copy



#from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
#plt.ion()  

from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

In [2]:
torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device(type='cuda', index=0)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
# Change to fit hardware
BATCH_SIZE = 64
print_every = 50
MODEL = 'alexnet'

In [5]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/scratch/kv942/ccm_proj/datadir_act//'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

train_loader = DataLoader(image_datasets['train'],
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          )
val_loader = DataLoader(image_datasets['val'],
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         )
# test_loader = DataLoader(image_datasets['test'],
#                          batch_size=BATCH_SIZE,
#                          shuffle=False,
#                          )

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [6]:
dataset_sizes

{'train': 39427, 'val': 4487}

In [7]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [8]:
# inputs, classes = next(iter(dataloaders['train']))
# out = torchvision.utils.make_grid(inputs)
#imshow(out, title=[class_names[x] for x in classes])

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    # Training steps
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print("Epoch: ", epoch)
        model.train()
        
        running_loss = 0.0   
        running_corrects = 0
        number_training_steps = 0
        # Iterate over data.
        for batch_id, (inputs, labels) in enumerate(train_loader):
            
            inputs, labels = Variable(inputs).to(device), Variable(labels).to(device)
            outputs = model(inputs)
            model.zero_grad() #same as optimizer.zero_grad()
            
            loss = criterion(outputs, labels)
            running_loss += loss.item() # sum up loss of all minibatches
            
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data).cpu()
            
            loss.backward()
            optimizer.step()
            number_training_steps+=1
            
            if batch_id % print_every == 0:
                # report performance
                print('Train [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_id * len(inputs), len(train_loader.dataset),
                100. * batch_id / len(train_loader), loss.item()))

        scheduler.step()
        
        epoch_trainloss = running_loss / number_training_steps
        #train_loss_list.append(epoch_trainloss)
        epoch_trainaccuracy = running_corrects.double() / len(train_loader.dataset)
        #train_acc_list.append(epoch_trainaccuracy)
        
        # report performance
        print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        epoch_trainloss, running_corrects, len(train_loader.dataset),
        100. * epoch_trainaccuracy))
        
        
    
        # Evaluate after every epoch
        
        model.eval()
        running_loss = 0
        running_corrects = 0

        with torch.no_grad():
            
            number_val_steps = 0
            for batch_id, (inputs, labels) in enumerate(val_loader):
                inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))
                #print("labels = ", labels)
                outputs = model(inputs)
                #print("outputs = ", outputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() # sum up loss of all minibatches
                number_val_steps += 1
                
                _, preds = torch.max(outputs, 1)
                running_corrects += torch.sum(preds == labels.data).cpu()
                
            
            epoch_valloss = running_loss / number_val_steps
            #val_loss_list.append(epoch_valloss)
            epoch_valaccuracy = running_corrects.double() / len(val_loader.dataset)
            #val_acc_list.append(epoch_valaccuracy)
            #auc = roc_auc_score(truths, predictions)
            
            print('Validation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            epoch_valloss, running_corrects, len(val_loader.dataset),
            100. * epoch_valaccuracy))
    
        # deep copy the model
        if epoch_valaccuracy > best_acc:
                best_acc = epoch_valaccuracy
                best_model_wts = copy.deepcopy(model.state_dict())
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

# ResNet 50

In [11]:
model = torchvision.models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/kv942/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
for params in model.parameters():
    params.requires_grad = False

In [25]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

In [26]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [27]:
if torch.cuda.device_count() > 1:
    print("we have multiple GPUs..")
    model = nn.DataParallel(model)


In [28]:
model = model.to(device)

In [29]:
torch.cuda.empty_cache()

In [30]:
summary(model, input_size=(3, 224, 224), batch_size=BATCH_SIZE, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [64, 64, 112, 112]           9,408
       BatchNorm2d-2         [64, 64, 112, 112]             128
              ReLU-3         [64, 64, 112, 112]               0
         MaxPool2d-4           [64, 64, 56, 56]               0
            Conv2d-5           [64, 64, 56, 56]           4,096
       BatchNorm2d-6           [64, 64, 56, 56]             128
              ReLU-7           [64, 64, 56, 56]               0
            Conv2d-8           [64, 64, 56, 56]          36,864
       BatchNorm2d-9           [64, 64, 56, 56]             128
             ReLU-10           [64, 64, 56, 56]               0
           Conv2d-11          [64, 256, 56, 56]          16,384
      BatchNorm2d-12          [64, 256, 56, 56]             512
           Conv2d-13          [64, 256, 56, 56]          16,384
      BatchNorm2d-14          [64, 256,

In [31]:
num_epochs = 5

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [32]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,num_epochs=num_epochs)

Epoch:  0
Train [0/39427 (0%)]	Loss: 2.149530
Train [3200/39427 (8%)]	Loss: 1.483621
Train [6400/39427 (16%)]	Loss: 1.621048
Train [9600/39427 (24%)]	Loss: 1.366948
Train [12800/39427 (32%)]	Loss: 1.470694
Train [16000/39427 (41%)]	Loss: 1.161606
Train [19200/39427 (49%)]	Loss: 1.177349
Train [22400/39427 (57%)]	Loss: 1.286365
Train [25600/39427 (65%)]	Loss: 1.205082
Train [28800/39427 (73%)]	Loss: 1.063229
Train [32000/39427 (81%)]	Loss: 1.167223
Train [35200/39427 (89%)]	Loss: 1.086544
Train [38400/39427 (97%)]	Loss: 1.063488
Train set: Average loss: 1.3271, Accuracy: 20914/39427 (53.04%)
Validation set: Average loss: 1.7221, Accuracy: 1779/4487 (39.65%)

Epoch:  1
Train [0/39427 (0%)]	Loss: 1.143021
Train [3200/39427 (8%)]	Loss: 0.878929
Train [6400/39427 (16%)]	Loss: 1.042602
Train [9600/39427 (24%)]	Loss: 1.064815
Train [12800/39427 (32%)]	Loss: 1.292484
Train [16000/39427 (41%)]	Loss: 0.792833
Train [19200/39427 (49%)]	Loss: 1.152688
Train [22400/39427 (57%)]	Loss: 1.125247
Train

In [33]:
torch.save(model.state_dict(), "./model_results/AlexNet_train_lastlayer_ACT.pth")